# pydlem.prep Package Example
This notebook provides a complete example of acquiring meterology data and combining it with reservoir data to produce a formatted input data file for running the pydlem model.

First import necessary packages/modules

In [1]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np


pydlem_dir = r'C:\Users\CND905\Downloaded_Programs\pydlem\pydlem'
os.chdir(pydlem_dir)

from prep.datafile import CreateInputFile
from prep.lakegeom import area_from_eac_curve
from prep.lakegeom import calc_lake_depth
from prep.lakegeom import calc_fetch_length
from prep.metdata import calculate_vpd
from prep.metdata import get_gridmet_at_points
from prep.datafile import check_format

Initializing pydlem.prep module.


## Example 1: Static Reservoir Inputs
For this example, we will load the geometry data and extract GridMET meterology for the lakes of interest. In this case, a shapefile is used but this could be extracted from a database or any other method supported by Geopandas.

In [2]:
exres_pth = Path(r'C:\Users\CND905\Downloaded_Programs\pydlem\pydlem\examples\example_data\HWWY_reservoirs_5070.shp')
exres = gpd.read_file(exres_pth)
# this file is in crs 5070, need it to be 4326 for getting GridMET
exres = exres.to_crs(4326)

exres

,Permanent_,FDate,Resolution,GNIS_ID,GNIS_Name,AreaSqKm,Elevation,ReachCode,FType,FCode,Visibility,Shape_Leng,Shape_Area,NHDPlusID,VPUID,geometry
0,71772797,2003-02-20,2,01600849,Last Chance Reservoir,0.051000,0.0000,10090101001988,390,39004,0,0.010722,0.000006,2.300250e+13,1009,"POLYGON Z ((-107.20213 44.54076 0, -107.20223 ..."
1,71772129,2016-11-03,2,01606753,Colorado Colony Ditch Company Reservoir Number 2,1.257994,0.0000,10090101001985,390,39004,0,0.080258,0.000142,2.300250e+13,1009,"POLYGON Z ((-107.21702 44.54794 0, -107.21707 ..."
2,71772161,2003-02-20,2,01600017,Granger Reservoir,0.028000,0.0000,10090101001980,390,39004,0,0.008714,0.000003,2.300250e+13,1009,"POLYGON Z ((-107.2024 44.56734 0, -107.2023 44..."
3,71759682,2003-02-20,2,01606767,Dome Lake Number 1,0.204000,0.0000,10090101001957,390,39004,0,0.022006,0.000023,2.300250e+13,1009,"POLYGON Z ((-107.31638 44.60378 0, -107.3165 4..."
4,71772927,2003-02-20,2,01608139,Willits Reservoir,0.026000,0.0000,10090101001989,390,39004,0,0.007028,0.000003,2.300250e+13,1009,"POLYGON Z ((-107.19553 44.53306 0, -107.19525 ..."
5,71759808,2003-02-20,2,01587705,Dome Lake Reservoir,0.357000,0.0000,10090101001969,390,39004,0,0.031711,0.000040,2.300250e+13,1009,"POLYGON Z ((-107.30002 44.58735 0, -107.30028 ..."
6,71818121,2003-02-20,2,01604632,Sawmill Reservoir,0.265000,0.0000,10090101001947,390,39004,0,0.030454,0.000030,2.300250e+13,1009,"POLYGON Z ((-107.30803 44.62497 0, -107.30803 ..."
7,71759956,2003-02-20,2,01596266,Weston Reservoir,0.138000,0.0000,10090101001976,390,39004,0,0.019932,0.000016,2.300250e+13,1009,"POLYGON Z ((-107.26949 44.56849 0, -107.26952 ..."
8,120031076,2012-11-26,2,01599036,Cross Creek Reservoir,0.223523,2745.6384,10090101001994,390,39009,0,0.022913,0.000025,2.300250e+13,1009,"POLYGON Z ((-107.20593 44.5041 0, -107.20575 4..."
9,71772901,2016-11-03,2,01606391,Big Horn Reservoir,0.669102,0.0000,10090101001993,390,39004,0,0.042110,0.000076,2.300250e+13,1009,"POLYGON Z ((-107.20568 44.52121 0, -107.20608 ..."


The default behavior of the CreateInputFile class is to download a complete POR series of meteorology from GridMET. As of right now, if you want a shorter period, use the get_gridmet_at_points() function directly.

In [3]:
# This example provides a unique identifier column rather than using the default DataFrame index.
# We extract just 15 days of meterology data for this example. This function is slow for a long period of record over many sites.
exres_met = get_gridmet_at_points(exres, 'Permanent_', start='2018-01-01', end='2022-12-31')

C:\Users\CND905\Downloaded_Programs\pydlem\pydlem\prep\metdata.py:51: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = list(zip(in_geom.geometry.centroid.x, in_geom.geometry.centroid.y))


Retrieving GridMET cells...


C:\Users\CND905\Downloaded_Programs\pydlem\pydlem\prep\metdata.py:72: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gmt_cntrs = gmt_cells.drop_duplicates(subset='cell_id').centroid


8 unique GridMET cells found for 11 input features.
Fetching GridMET data for unique cells...


Cells: 100%|██████████| 8/8 [01:26<00:00, 10.82s/it]


In [4]:
exres_met

<xarray.Dataset> Size: 1MB
Dimensions:   (time: 1826, location: 11)
Coordinates:
    lat       (location) float64 88B 44.54 44.56 44.57 ... 44.5 44.53 44.55
    lon       (location) float64 88B -107.2 -107.2 -107.2 ... -107.2 -107.2
    elev      (location) float64 88B 2.638e+03 2.518e+03 ... 2.664e+03 2.618e+03
  * location  (location) <U9 396B '71772797' '71772129' ... '71772619'
  * time      (time) datetime64[ns] 15kB 2018-01-01 2018-01-02 ... 2022-12-31
Data variables:
    precip    (time, location) float64 161kB 0.0 0.0 0.0 0.0 ... 0.3 0.3 0.45
    min_temp  (time, location) float64 161kB 255.4 255.7 256.1 ... 261.2 261.6
    max_temp  (time, location) float64 161kB 269.0 269.3 269.2 ... 271.4 271.9
    solrad    (time, location) float64 161kB 79.8 79.3 80.4 ... 59.35 58.9 58.5
    wind_dir  (time, location) float64 161kB 304.0 304.0 304.0 ... 294.0 294.0
    wind_vel  (time, location) float64 161kB 4.05 3.9 4.2 3.5 ... 3.05 3.25 3.4
    vpd       (time, location) float64 161kB 0.175 0.18 0.18 ... 0.11 0.125 0.14
Attributes:
    featureType:  timeSeries

To demonstrate loading a netcdf into the CreateInputFile class, we will save this meteorology data as a netcdf and then create an input datafile from it.

In [5]:
pydlem_dir = Path(r'C:\Users\CND905\Downloaded_Programs\pydlem\pydlem\examples\example_data')
os.chdir(pydlem_dir)
exres_met.to_netcdf(pydlem_dir / 'prep_Example1_met.nc')

Lake Area and Depth are required inputs to create the datafile, as pydlem needs these to run the computations. So first, we will create these and then load inputs from our previos meteorlogy netcdf file. The easiest way to start with pydlem is to assume static values for reservoir variables 'Depth' and 'Surface Area.' We will create some static data for the 15 days of interest.

In [6]:
# surface area from NHD
lareas = np.array([exres[exres['Permanent_'] == x]['AreaSqKm'].iloc[0] for x in exres_met.location.values])
lareas_arr = np.tile(lareas, (len(exres_met.time + 1), 1))
# in this case we will make it an xarray DataArray
Lake_Area = xr.DataArray(lareas_arr, coords=exres_met.coords, attrs={'standard_name': 'Lake Surface Area', 'units': 'km^2'})
Lake_Area.name = 'LakeArea'

# Next, to get depth we need volume. You can estimate the volume or assume an average value for depth. In this case, the reservoirs have max capacity 
# data that are manually listed here and used to calcuate depth.

# These are in cubic kilometers
caps = np.array([0.000259, 0.0128, 0.00018, 0.0042, 0.000099, 0.00186, 0.00157, 0.00046, 0.00098, 0.0057, 0.00069])
caps_arr = np.tile(caps, (len(exres_met.time + 1), 1))
ldpth = (caps_arr / lareas_arr) * 1000 # convert to meters of depth
Lake_Depth = xr.DataArray(ldpth, coords=exres_met.coords, attrs={'standard_name': 'Lake Depth', 'units': 'm'})
Lake_Depth.name = 'LakeDepth'

Now we load our saved netcdf from file into xarray and create the input data file.

In [7]:
met_ff = xr.load_dataset(r'C:\Users\CND905\Downloaded_Programs\pydlem\pydlem\examples\example_data\prep_Example1_met.nc')
nw_Idat = CreateInputFile(geoms=None, lake_area=Lake_Area, lake_depth=Lake_Depth, met_data=met_ff, met_source='from_file')

New variable added.
New variable added.


We still are missing some required variables. mean_temp can be calculated from our GridMET derived min/max temperature and added to the dataset. 'ftch_len' is the FETCH for the individual lakes. This is an important term in the dlem model that accounts for wind blowing across the lakes surface. The lakegeom.py module within the pydlem.prep package has a method to calculate fetch length needed to run the model.

<u>The mean temp is also calculated in the model.py code (line 57) and there is no code available for inputting the mean_temp if calculated beforehand. So, this calculation and adding variable is not necessary.</u>

Next we will compute these 2 required variables and add them to the dataset using the class method add_variable().

In [8]:
# mean temp
# mean_temp = ((nw_Idat.data.min_temp + nw_Idat.data.max_temp) / 2) - 273.15  # also convert to Celcius from GridMET native Kelvin
# nw_Idat.add_variable(mean_temp, variable_name='mean_temp', var_attrs={'standard_name': 'Average Daily Temperature', 'units': 'Celcius'})

In [9]:
# calculating the fetch length
# fetch is very dynamic, it depends on lake area, wind direction, and surface area geometry which are always flucuating
# thus, the fetch length function calculates only one fetch length at a time, so input arrays need to be looped over their 2 dimensions
# (time, location) calculating fetch length for all values

# assign necessary variables and complete any conversions
rows = nw_Idat.data.wind_dir.shape[0]
cols = nw_Idat.data.wind_dir.shape[1]
wnd_d = nw_Idat.data.wind_dir.values
lk_A = nw_Idat.data.LakeArea.values * 1000.0**2  # convert to meters
# Loop through data arrays of wind direction for each timestamp(x) and location(y)
resvs = exres.to_crs(exres.estimate_utm_crs())
xs = []
for x in range(0, rows):
    ys = []
    for y in range(0, cols):
        gmtry = resvs.loc[resvs['Permanent_'] == nw_Idat.data.location[y].item()].geometry
        wnd = wnd_d[x, y]
        A = lk_A[x, y]
        ftchL = calc_fetch_length(gmtry, wnd, A)
        ys.append(ftchL)
    xs.append(ys)
# create xarray DataArray from the resulting ndarray
flen = np.array(xs)
flen = xr.DataArray(flen, coords=nw_Idat.data.coords, attrs={'standard_name': 'Wind Fetch Length', 'units': 'm'})
# Add to the dataset
nw_Idat.add_variable(flen, 'ftch_len')

New variable added.


All variables required by the model are added. lrad can be estimated in the model but is not required. Changes to the shortwave readiation were also made so the model code can be imported as 'solrad' instead of srad.
As stated above, mean_temp is not required and can be removed from check_format function.  

In [10]:
check_format(nw_Idat.data)

MISSING VARIABLES
lrad
All necessary coordinates exist and are labeled properly


C:\Users\CND905\Downloaded_Programs\pydlem\pydlem\prep\datafile.py:194: UserWarning: There are missing or mislabeled variables in the dataset. See the following:
  warnings.warn("There are missing or mislabeled variables in the dataset. See the following:")


In [11]:
nw_Idat.data

<xarray.Dataset> Size: 2MB
Dimensions:    (time: 1826, location: 11)
Coordinates:
    lat        (location) float64 88B 44.54 44.56 44.57 ... 44.5 44.53 44.55
    lon        (location) float64 88B -107.2 -107.2 -107.2 ... -107.2 -107.2
    elev       (location) float64 88B 2.638e+03 2.518e+03 ... 2.618e+03
  * location   (location) <U9 396B '71772797' '71772129' ... '71772619'
  * time       (time) datetime64[ns] 15kB 2018-01-01 2018-01-02 ... 2022-12-31
Data variables:
    precip     (time, location) float64 161kB 0.0 0.0 0.0 0.0 ... 0.3 0.3 0.45
    min_temp   (time, location) float64 161kB 255.4 255.7 256.1 ... 261.2 261.6
    max_temp   (time, location) float64 161kB 269.0 269.3 269.2 ... 271.4 271.9
    solrad     (time, location) float64 161kB 79.8 79.3 80.4 ... 59.35 58.9 58.5
    wind_dir   (time, location) float64 161kB 304.0 304.0 304.0 ... 294.0 294.0
    wind_vel   (time, location) float64 161kB 4.05 3.9 4.2 3.5 ... 3.05 3.25 3.4
    vpd        (time, location) float64 161kB 0.175 0.18 0.18 ... 0.125 0.14
    LakeArea   (time, location) float64 161kB 0.051 1.258 0.028 ... 0.6691 0.12
    LakeDepth  (time, location) float64 161kB 5.078 10.17 6.429 ... 8.519 5.75
    ftch_len   (time, location) float64 161kB 154.7 535.2 125.4 ... 412.8 208.9
Attributes:
    featureType:  timeSeries

We now have a successfully built input file for pydlem. We can save it as Example 1 for use in the model running example notebook.

In [12]:
nw_Idat.save_datafile('Example1_static_inputs.nc')